In [1]:
%fcflags: -w
%module: mod_arrays
module mod_arrays

  ! Utility functions that operate on arrays.

  implicit none

  private
  public :: argsort, average, crossneg, crosspos, intdate, moving_average,&
            moving_std, reverse, std

contains

  pure function argsort(x) result(a)
    ! Returns indices that sort x from low to high.
    real, intent(in):: x(:)
    integer :: a(size(x))
    integer :: i, i0, tmp1
    real :: tmp2
    real :: xwork(size(x))
    a = [(real(i), i = 1, size(x))]
    xwork = x
    do i = 1, size(x) - 1
      i0 = minloc(xwork(i:), 1) + i - 1
      if (i0 /= i) then
        tmp2 = xwork(i)
        xwork(i) = xwork(i0)
        xwork(i0) = tmp2
        tmp1 = a(i)
        a(i) = a(i0)
        a(i0) = tmp1
      end if
    end do
  end function argsort

  pure real function average(x)
    ! Returns a average of x.
    real, intent(in) :: x(:)
    average = sum(x) / size(x)
  end function average

  pure function crossneg(x, w) result(res)
    ! Returns indices where input array x crosses its
    ! moving average with window w from positive to negative.
    real, intent(in) :: x(:)
    integer, intent(in) :: w
    integer, allocatable :: res(:)
    real, allocatable :: xavg(:)
    logical, allocatable :: greater(:), smaller(:)
    integer :: i
    res = [(i, i = 2, size(x))]
    xavg = moving_average(x, w)
    greater = x > xavg
    smaller = x < xavg
    res = pack(res, smaller(2:) .and. greater(:size(x)-1))
  end function crossneg

  pure function crosspos(x, w) result(res)
    ! Returns indices where input array x crosses its
    ! moving average with window w from negative to positive.
    real, intent(in) :: x(:)
    integer, intent(in) :: w
    integer, allocatable :: res(:)
    real, allocatable :: xavg(:)
    logical, allocatable :: greater(:), smaller(:)
    integer :: i
    res = [(i, i = 2, size(x))]
    xavg = moving_average(x, w)
    greater = x > xavg
    smaller = x < xavg
    res = pack(res, greater(2:) .and. smaller(:size(x)-1))
  end function crosspos

  pure elemental integer function intdate(t)
    ! Converts a time stamp in format YYYY-mm-dd to integer.
    character(len=10), intent(in) :: t
    character(len=8) :: str
    str = t(1:4) // t(6:7) // t(9:10)
    read(str, *) intdate
  end function intdate

  pure function moving_average(x, w) result(res)
    ! Returns the moving average of x with one-sided window w.
    real, intent(in) :: x(:)
    integer, intent(in) :: w
    real :: res(size(x))
    integer :: i, i1
    do i = 1, size(x)
      i1 = max(i-w, 1)
      res(i) = average(x(i1:i))
    end do 
  end function moving_average

  pure function moving_std(x, w) result(res)
    ! Returns the moving standard deviation of x with one-sided window w.
    real, intent(in) :: x(:)
    integer, intent(in) :: w
    real :: res(size(x))
    integer :: i, i1
    do i = 1, size(x)
      i1 = max(i-w, 1)
      res(i) = std(x(i1:i))
    end do 
  end function moving_std

  pure function reverse(x)
    ! Reverses the order of elements of x.
    real, intent(in) :: x(:)
    real :: reverse(size(x))
    reverse = x(size(x):1:-1)
  end function reverse

  pure real function std(x)
    ! Returns the standard deviation of x.
    real, intent(in) :: x(:)
    std = sqrt(average((x - average(x))**2))
  end function std

end module mod_arrays

[gfort kernel] module objects created successfully: mod_arrays.o

In [2]:
%fcflags: -w
%module: mod_alloc
module mod_alloc

  implicit none

  private
  public :: alloc, free

contains

  subroutine alloc(a, n)
    real, allocatable, intent(in out) :: a(:)
    integer, intent(in) :: n
    integer :: stat
    character(len=100) :: errmsg
    if (allocated(a)) call free(a)
    allocate(a(n), stat=stat, errmsg=errmsg)
    if (stat > 0) error stop errmsg
  end subroutine alloc

  subroutine free(a)
    real, allocatable, intent(in out) :: a(:)
    integer :: stat
    character(len=100) :: errmsg
    if (.not. allocated(a)) return
    deallocate(a, stat=stat, errmsg=errmsg)
    if (stat > 0) error stop errmsg
  end subroutine free

end module mod_alloc

[gfort kernel] module objects created successfully: mod_alloc.o

In [3]:
%module: mod_io
module mod_io

  ! A helper module for parsing stock price data in csv format.

  use mod_alloc, only: alloc

  implicit none

  private
  public :: read_stock, write_stock

contains

  integer function num_records(filename)
    ! Return the number of records (lines) of a text file.
    character(len=*), intent(in) :: filename
    integer :: fileunit
    open(newunit=fileunit, file=filename)
    num_records = 0
    do
      read(unit=fileunit, fmt=*, end=1)
      num_records = num_records + 1
    end do
    1 continue
    close(unit=fileunit)
  end function num_records

  subroutine read_stock(filename, time, open, high, low, close, adjclose, volume)
    ! Read daily stock prices from a csv file.
    character(len=*), intent(in) :: filename
    character(len=:), allocatable, intent(in out) :: time(:)
    real, allocatable, intent(in out) :: open(:), high(:), low(:),&
                                         close(:), adjclose(:), volume(:)
    integer :: fileunit, n, nm
    nm = num_records(filename) - 1
    if (allocated(time)) deallocate(time)
    allocate(character(len=10) :: time(nm))
    call alloc(open, nm)
    call alloc(high, nm)
    call alloc(low, nm)
    call alloc(close, nm)
    call alloc(adjclose, nm)
    call alloc(volume, nm)
    open(newunit=fileunit, file=filename)
    read(fileunit, fmt=*, end=1)
    do n = 1, nm
      read(fileunit, fmt=*, end=1) time(n), open(n),&
        high(n), low(n), close(n), adjclose(n), volume(n)
    end do
    1 close(fileunit)
  end subroutine read_stock

  subroutine write_stock(filename, time, price, mvavg, mvstd)
    ! Write derived stock data to file.
    character(len=*), intent(in) :: filename
    character(len=:), allocatable, intent(in) :: time(:)
    real, intent(in) :: price(:), mvavg(:), mvstd(:)
    integer :: fileunit, n
    open(newunit=fileunit, file=filename)
    do n = 1, size(time)
      write(fileunit, fmt=*) time(n), price(n), mvavg(n), mvstd(n)
    end do
    close(fileunit)
  end subroutine write_stock 

end module mod_io

[gfort kernel] module objects created successfully: mod_io.o

In [4]:
%fcflags: mod_arrays.o mod_io.o mod_alloc.o
program stock_gain

    use mod_arrays, only : reverse
    use mod_io,     only : read_stock
    
    implicit none
    
    character(len=4), allocatable :: symbols(:)
    character(len=:), allocatable :: time   (:)
    
    real, allocatable :: open(:), high(:), low(:), &
    close(:), adjclose(:), volume(:)
    
    integer :: n
    real    :: gain
    
    symbols = ['AAPL', 'AMZN', 'CRAY', 'CSCO', 'HPQ ', 'IBM ', & 
               'INTC', 'MSFT', 'NVDA', 'ORCL']
               
    do n = 1, size(symbols)
    
        call read_stock('data/' // trim(symbols(n)) // '.csv', time, open, high, low, close, adjclose, volume)
        
        adjclose = reverse(adjclose)
        gain = (adjclose(size(adjclose)) - adjclose(1))
        
        if (n == 1) then
            print *, &
              time(size(time)) // ' through ' // time(1)
            print *, 'Symbol, Gain (USD), Relative gain (%)'
            print *, '*************************************'
        end if
        
        print *, symbols(n), gain, &
          nint(gain / adjclose(1) * 100)
          
  end do
  
end program stock_gain

 2000-01-03 through 2018-05-14
 Symbol, Gain (USD), Relative gain (%)
 *************************************
 AAPL   184.594589            5192
 AMZN   1512.16003            1692
 CRAY   9.60000038              56
 CSCO   1.71649933               4
 HPQ    1.55270004               7
 IBM    60.9193039              73
 INTC   25.8368015              89
 MSFT   59.4120979             154
 NVDA   251.745300            6964
 ORCL   20.3501987              77
